In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import graph_generator as gg
import algorithms as dp
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
from gnn_library.util import train, save, load
from evaluate import batched_test_model

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

In [ ]:
args = {
    'processor':         'GENConv',
    'head':              'classification',
    'num_layers':        2,
    'num_mlp_layers':    2,
    'aggr':              'max',
    'batch_size':        32,
    'node_feature_dim':  4,
    'edge_feature_dim':  1,
    'graph_feature_dim': 2,
    'hidden_dim':        64,
    'dropout':           0.35,
    'epochs':            50,
    'opt':               'adam',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.0001,
    'device':            device
}

m = 10; n = 6; train_num = 100; test_num = 30

er_config = {
    'graph_type': 'ER',
    'p': 1,
    'weighted': True
}
ba_config = {
    'graph_type': 'BA',
    'ba_param': 4,
    'weighted': True
}
geom_config = {
    'graph_type': 'GEOM',
    'threshold': 0.2,
    'scaling': 1 / np.sqrt(2)
}

class Dataset(InMemoryDataset):

    def __init__(self, data_list):
        super().__init__(None)
        self.data, self.slices = self.collate(data_list)


train_dataset_1 = Dataset(
    [
        *gg.generate_examples(train_num, 10, 6, np.random.uniform(0.5, 1, m), args['head'], **er_config),
        *gg.generate_examples(train_num, 10, 6, np.random.uniform(0.5, 1, m), args['head'], **ba_config),
        *gg.generate_examples(2 * train_num, 10, 6, np.random.uniform(0.5, 1, m), args['head'], **geom_config),
    ]
)

test_dataset_1 = Dataset(
    [
        *gg.generate_examples(test_num, 10, 6, np.random.uniform(0.5, 1, m), args['head'], **er_config),
        *gg.generate_examples(test_num, 10, 6, np.random.uniform(0.5, 1, m), args['head'], **ba_config),
        *gg.generate_examples(2 * test_num, 10, 6, np.random.uniform(0.5, 1, m), args['head'], **geom_config)
    ]
)

train_dataset_2 = Dataset(
    [
        *gg.generate_examples(train_num, 8, 8, np.random.uniform(0.5, 1, m), args['head'], **er_config),
        *gg.generate_examples(train_num, 8, 8, np.random.uniform(0.5, 1, m), args['head'], **ba_config),
        *gg.generate_examples(2 * train_num, 8, 8, np.random.uniform(0.5, 1, m), args['head'], **geom_config),
    ]
)

test_dataset_2 = Dataset(
    [
        *gg.generate_examples(test_num, 8, 8, np.random.uniform(0.5, 1, m), args['head'], **er_config),
        *gg.generate_examples(test_num, 8, 8, np.random.uniform(0.5, 1, m), args['head'], **ba_config),
        *gg.generate_examples(2 * test_num, 8, 8, np.random.uniform(0.5, 1, m), args['head'], **geom_config)
    ]
)

train_loader_1 = DataLoader(
    train_dataset_1,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=16
)

test_loader_1 = DataLoader(
    test_dataset_1,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=16
)

train_loader_2 = DataLoader(
    train_dataset_2,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=4
)

test_loader_2 = DataLoader(
    test_dataset_2,
    batch_size=args['batch_size'],
    shuffle=True,
    num_workers=4
)



In [ ]:
_, _, GNN1, _ = train(train_loader_1, test_loader_1, args)
#_, _, GNN2, _ = train(train_loader_2, test_loader_2, args)

In [ ]:
save(GNN1, args, 'SKIP_CLASS')
# save(GNN2, args, 'GNN2')

In [ ]:
model1, args = load('SKIP_CLASS')
#model2 = load('GNN2')

In [ ]:
num_trials = 10
batch_size = 5
node_config = (48, 16)
graph_config =  {
    'graph_type': 'ER',
    'p': 0.75,
    'weighted': True
}

(gnn1_learned_ratios, greedy_ratios1), log1 = batched_test_model(
            model1,
            args,
            num_trials,
            batch_size,
            *node_config,
            graph_config)

In [ ]:
print(np.mean(gnn1_learned_ratios), np.mean(greedy_ratios1))

In [ ]:
num_trials = 250
batch_size = 50
node_config = (48, 16)
graph_config =  {
    'graph_type': 'ER',
    'p': 0.75,
    'weighted': True
}

(gnn1_learned_ratios, greedy_ratios1), log1 = batched_test_model(
            model1,
            device,
            num_trials,
            batch_size,
            *node_config,
            graph_config)

(gnn2_learned_ratios, greedy_ratios2), log2 = batched_test_model(
            model2,
            device,
            num_trials,
            batch_size,
            *node_config,
            graph_config)

In [ ]:
for key, val in log1.items():
    print(key, np.mean(val))

In [ ]:
for key, val in log2.items():
    print(key, np.mean(val))

In [ ]:
print(np.mean(gnn1_learned_ratios), np.mean(greedy_ratios1))
print(np.mean(gnn2_learned_ratios), np.mean(greedy_ratios2))

In [ ]:
num_trials = 500
node_configs = [(x, 16) for x in np.arange(4, 64, 2)]
# of nodes [20 -> 80]
# of nodes in batch [10,000 -> 40,000]
batch_size = 100 #[int(min(32, x + y)) for (x, y) in node_configs]
graph_configs = [
    # {
    #     'graph_type': 'GM'
    # },
    {
        'graph_type': 'ER',
        'p': 0.75,
        'weighted': True
    },
    {
        'graph_type': 'BA',
        'ba_param': 4,
        'weighted': True
    },
    {
        'graph_type': 'GEOM',
        'threshold': 0.2,
        'scaling': 1 / np.sqrt(2),
    }
]

ratios = [x/y for (x,y) in node_configs]
print(ratios)
data = {config['graph_type']: [] for config in graph_configs}
for graph_config in graph_configs:
    for i, node_config in enumerate(node_configs):
        (gnn1_learned_ratios, greedy_ratios), _ = batched_test_model(
            GNN1,
            device,
            num_trials,
            batch_size,
            *node_config,
            graph_config)

        (gnn2_learned_ratios, _), _ = batched_test_model(
            GNN2,
            device,
            num_trials,
            batch_size,
            *node_config,
            graph_config)

        data[graph_config['graph_type']].append(np.array(
            [gnn1_learned_ratios, gnn2_learned_ratios, greedy_ratios]
        ))

In [ ]:
for graph_type, comp_ratios in data.items():
    greedy_avg_ratios = []
    gnn1_avg_ratios = []
    gnn2_avg_ratios = []
    max_avg_ratios = []

    for trial_ratios in comp_ratios:
        gnn1_avg_ratios.append(np.array(trial_ratios[0]).mean())
        gnn2_avg_ratios.append(np.array(trial_ratios[1]).mean())
        greedy_avg_ratios.append(np.array(trial_ratios[2]).mean())
        max_avg_ratios.append(np.array(np.max(trial_ratios, axis=0)).mean())

    print(graph_type)
    fig = plt.figure(figsize=(8,6))
    plt.title(graph_type)
    plt.plot(ratios, gnn1_avg_ratios, label='GNN1')
    plt.plot(ratios, gnn2_avg_ratios, label='GNN2')
    plt.plot(ratios, greedy_avg_ratios, label='Greedy')
    plt.plot(ratios, max_avg_ratios, label='MAX')
    plt.xlabel('# online / # offline')
    plt.ylabel('Average competitive ratio')
    plt.legend()
    plt.show()


  

In [ ]:
num_trials = 250
node_configs = [(32, x)) for x in np.arange(4, 33)]
graph_configs = [
    # {
    #     'graph_type': 'GM'
    # },
    {
        'graph_type': 'ER',
        'p': 0.75,
        'weighted': True
    },
    # {
    #     'graph_type': 'BA',
    #     'ba_param': 4,
    #     'weighted': True
    # },
    # {
    #     'graph_type': 'GEOM',
    #     'threshold': 0.2,
    #     'scaling': 1 / np.sqrt(2),
    # }
]

output = {}
# graph_config = {
#     'graph_type': 'GM',
# }
for p in [0.05, 0.1, 0.15, 0.2]:
    graph_config = {
        'graph_type': 'ER',
        'weighted': True,
        'p': p
    }
    for node_config in node_configs:
        out = test_model(trained_model, num_trials, *node_config, graph_config)
        #output[f"{graph_config['graph_type']}, {node_config}"] = out
        output[f"{graph_config['graph_type']}(p={p}), {node_config}"] = out

In [ ]:
from matplotlib.ticker import AutoMinorLocator
def format_dict(vals):
   return dict(zip(
            ['label', 'whislo', 'q1', 'med', 'q3', 'whishi'],
            vals
        ))

KHALIL = {
  'ER(p=0.05), (30, 10)': format_dict(['Khalil', 0.66, 0.84, 0.895, 0.945, 1.0]),
  'ER(p=0.1), (30, 10)': format_dict(['Khalil', 0.69, 0.83, 0.88, 0.93, 1.0]),
  'ER(p=0.15), (30, 10)': format_dict(['Khalil', 0.71, 0.84, 0.89, 0.935, 1.0]),
  'ER(p=0.2), (30, 10)': format_dict(['Khalil', 0.735, 0.855, 0.905, 0.94, 1.0]),
  'ER(p=0.05), (60, 10)': format_dict(['Khalil', 0.685, 0.83, 0.88, 0.925, 1.0]),
  'ER(p=0.1), (60, 10)': format_dict(['Khalil', 0.75, 0.86, 0.905, 0.935, 1.0]),
  'ER(p=0.15), (60, 10)': format_dict(['Khalil', 0.79, 0.885, 0.93, 0.955, 1.0]),
  'ER(p=0.2), (60, 10)': format_dict(['Khalil', 0.835, 0.91, 0.945, 0.965, 1.0]),
  'ER(p=0.05), (100, 100)': format_dict(['Khalil', 0.83, 0.865, 0.875, 0.89, 0.94]),
  'ER(p=0.1), (100, 100)': format_dict(['Khalil', 0.84, 0.87, 0.88, 0.895, 0.93]),
  'ER(p=0.15), (100, 100)': format_dict(['Khalil', 0.86, 0.885, 0.905, 0.91, 0.955]),
  'ER(p=0.2), (100, 100)': format_dict(['Khalil', 0.87, 0.905, 0.92, 0.935, 0.965]),
  'GM, (30, 10)': format_dict(['Khalil', 0.7, 0.835, 0.87, 0.91, 1.0]),
  'GM, (60, 10)': format_dict(['Khalil', 0.77, 0.865, 0.90, 0.915, 1.0]),
  'GM, (100, 100)': format_dict(['Khalil', 0.83, 0.88, 0.89, 0.905, 0.94]),
}

boxprops = dict(linestyle='-', linewidth=2, color='black')
medianprops = dict(linestyle='-', linewidth=2, color='blue')
for title, stats in output.items():
    _, ax = plt.subplots(figsize=(6, 6))
    ax.bxp([KHALIL[title], *stats], showfliers=False, boxprops=boxprops, medianprops=medianprops)
    plt.title(title)
    # Give plot a gray background like ggplot.
    ax.set_facecolor('#EBEBEB')
    # Remove border around plot.
    [ax.spines[side].set_visible(False) for side in ax.spines]
    # Style the grid.
    ax.grid(which='major', color='white', linewidth=1.2)
    ax.grid(which='minor', color='white', linewidth=0.6)
    # Show the minor ticks and grid.
    ax.minorticks_on()
    # Now hide the minor ticks (but leave the gridlines).
    ax.tick_params(which='minor', bottom=False, left=False)

    # Only show minor gridlines once in between major gridlines.
    ax.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(AutoMinorLocator(2))
    plt.show()